# Training with PyTorch
This notebook shows how to train a PyTorch model using images from `vl-datasets`. 

<!--<badge>--><a href="https://colab.research.google.com/github/visual-layer/vl-datasets/blob/main/notebooks/train-pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->
<!--<badge>--><a href="https://kaggle.com/kernels/welcome?src=https://github.com/visual-layer/vl-datasets/blob/main/notebooks/train-pytorch.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/></a><!--</badge>-->

## Installation & Setting Up

In [1]:
!pip install vl-datasets -Uq

In [2]:
import vl_datasets
vl_datasets.__version__

'0.0.11'

## Download the VL food-101 Dataset

![datasetimage](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/static/img/food-101.jpg)

A challenging data set of 101 food categories is introduced, consisting of 101,000 images. Each class includes 250 manually reviewed test images and 750 training images. The training images deliberately retain some noise, primarily intense colors and occasional incorrect labels. All images have been rescaled to a maximum side length of 512 pixels.


The easiest way to get the clean version of food-101 dataset is to use `vl_datasets` package.

In [3]:
from vl_datasets import VLFood101

train_dataset = VLFood101('./', split='train')
valid_dataset = VLFood101('./', split='test')

Excluding churros/1440917.jpg from the train set
Excluding churros/1944265.jpg from the train set
Excluding churros/3867252.jpg from the train set
Excluding churros/644700.jpg from the train set
Excluding hot_and_sour_soup/1674538.jpg from the train set
Excluding hot_and_sour_soup/3918910.jpg from the train set
Excluding samosa/1727503.jpg from the train set
Excluding samosa/2754961.jpg from the train set
Excluding samosa/43271.jpg from the train set
Excluding samosa/529285.jpg from the train set
Excluding samosa/652074.jpg from the train set
Excluding samosa/918899.jpg from the train set
Excluding samosa/987023.jpg from the train set
Excluding sashimi/182979.jpg from the train set
Excluding sashimi/2160399.jpg from the train set
Excluding sashimi/241368.jpg from the train set
Excluding spring_rolls/1745022.jpg from the train set
Excluding spring_rolls/182658.jpg from the train set
Excluding spring_rolls/3149523.jpg from the train set
Excluding spring_rolls/3627865.jpg from the train s

Excluding eggs_benedict/1830519.jpg from the train set
Excluding eggs_benedict/2748311.jpg from the train set
Excluding cup_cakes/1005580.jpg from the train set
Excluding cup_cakes/1082593.jpg from the train set
Excluding cup_cakes/112438.jpg from the train set
Excluding cup_cakes/2219167.jpg from the train set
Excluding cup_cakes/2590269.jpg from the train set
Excluding cup_cakes/451074.jpg from the train set
Excluding cup_cakes/63497.jpg from the train set
Excluding takoyaki/537390.jpg from the train set
Excluding chocolate_mousse/1653769.jpg from the train set
Excluding chocolate_mousse/1734966.jpg from the train set
Excluding chocolate_mousse/2177988.jpg from the train set
Excluding chocolate_mousse/2616372.jpg from the train set
Excluding chocolate_mousse/343137.jpg from the train set
Excluding chocolate_mousse/766461.jpg from the train set
Excluding breakfast_burrito/2182358.jpg from the train set
Excluding breakfast_burrito/2428601.jpg from the train set
Excluding breakfast_burr

View the first five problematic images that are listed in the `.csv` file.

In [4]:
print(train_dataset.excluded_files[:5])

['churros/1440917.jpg', 'churros/1944265.jpg', 'churros/3867252.jpg', 'churros/644700.jpg', 'hot_and_sour_soup/1674538.jpg']


## Import PyTorch and Torchvision

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision

We can also view the exclude files with:

In [6]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=True)

Adjust the `batch_size` to a value that fits your hardware.

## Define the model architecture
Let's construct a basic convolutional model, `Resnet18` from `Torchvision`.

In [7]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

## Define the loss function and optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train the model
Now, let's write a simple training loop to train the model for 5 epochs on a GPU or CPU.

In [9]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader)}")


Using device: cuda
Epoch 1 - Loss: 2.3852627936056106
Epoch 2 - Loss: 1.816492844032029
Epoch 3 - Loss: 1.6049140736208123
Epoch 4 - Loss: 1.4772287748627744
Epoch 5 - Loss: 1.3921064696069492


## Evaluate the model
Finally we evaluate the model on the validation set and prints it's accuracy.

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for data in valid_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}")


Accuracy: 73.16210091050057
